In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import *  

df = spark.readStream\
        .format("cloudFiles")\
        .option("cloudFiles.format", "csv")\
        .option("readChangeFeed", "true")\
        .option("cloudFiles.schemaLocation", "/Volumes/payroll_catalog/landing/_schemas/bronze_offcycle")\
        .load("/Volumes/payroll_catalog/landing/offcycle")\
        .withColumn("ingestion_timestamp", current_timestamp())

query = df.writeStream\
    .format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "/Volumes/payroll_catalog/bronze/_checkpoints/bronze_offcycle")\
    .trigger(availableNow=True)\
    .table("payroll_catalog.bronze.bronze_offcycle")



In [0]:

spark.sql("""ALTER TABLE payroll_catalog.bronze.bronze_offcycle SET TBLPROPERTIES (delta.enableChangeDataFeed = true)""")

In [0]:
for stream in spark.streams.active:
    stream.stop()